# StateLog × TradeLog 解析テンプレート
- `analysis/env_data` と `analysis/bt_results` に配置した CSV を読み込み、環境指標とBT結果を突き合わせるためのノートです。
- `merged_trades` DataFrame を作成しておけば、帯域別・時間別の集計を容易に反復できます。

In [1]:
import pandas as pd
from pathlib import Path
from functools import lru_cache
from IPython.display import display, Markdown

def find_workspace_root(marker='analysis'):
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        if (candidate / marker).is_dir():
            return candidate
    raise FileNotFoundError("'{}' directory not found from {}".format(marker, current))

WORKSPACE_ROOT = find_workspace_root()
DATA_ROOT = WORKSPACE_ROOT / 'analysis'
ENV_DIR = DATA_ROOT / 'env_data'
BT_DIR = DATA_ROOT / 'bt_results'

def list_csv(directory, pattern='*.csv'):
    return sorted(directory.glob(pattern))

print('Workspace root:', WORKSPACE_ROOT)
print('Env dir:', ENV_DIR)
print('BT dir:', BT_DIR)
print('Detected env csv:', len(list_csv(ENV_DIR)))
print('Detected bt csv:', len(list_csv(BT_DIR, 'TradeLog_*.csv')))

def resolve_bt_path(bt_name=None, pattern='TradeLog_*.csv'):
    if bt_name:
        candidate = BT_DIR / bt_name
        if candidate.exists():
            return candidate
        raise FileNotFoundError(f"{candidate} が存在しません")
    files = list_csv(BT_DIR, pattern)
    if not files:
        raise FileNotFoundError(f"BT_DIR に {pattern} が見つかりません")
    return files[-1]

def load_bt_dataframe(bt_name=None):
    bt_path = resolve_bt_path(bt_name)
    df = pd.read_csv(bt_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('.', '-', regex=False))
    return df, bt_path

@lru_cache(maxsize=1)
def build_env_index():
    env_map = {}
    for csv_path in list_csv(ENV_DIR, '*.csv'):
        digits = ''.join(filter(str.isdigit, csv_path.stem))
        if len(digits) < 8:
            continue
        key = digits[-8:]
        env_map.setdefault(key, []).append(csv_path)
    return {k: tuple(v) for k, v in env_map.items()}

def load_env_frames(date_keys, sep=';'):
    env_map = build_env_index()
    frames = []
    missing = []
    for key in date_keys:
        paths = env_map.get(key)
        if not paths:
            missing.append(key)
            continue
        day_frames = [pd.read_csv(path, sep=sep) for path in paths]
        env_part = pd.concat(day_frames, ignore_index=True)
        env_part['bar_time'] = pd.to_datetime(env_part['bar_time'].str.replace('.', '-', regex=False))
        frames.append(env_part)
    return frames, missing

def concat_env_frames(frames):
    if not frames:
        raise ValueError('frames is empty')
    return pd.concat(frames, ignore_index=True).sort_values('bar_time')

def merge_entries_with_env(bt_df, env_df, tolerance='5min'):
    entries = bt_df[bt_df['event'] == 'ENTRY'].copy().sort_values('timestamp')
    env_sorted = env_df.sort_values('bar_time')
    merged = pd.merge_asof(entries, env_sorted, left_on='timestamp', right_on='bar_time',
                           direction='nearest', tolerance=pd.Timedelta(tolerance))
    return merged

def build_entry_exit(bt_df, merged):
    required = {'ticket', 'strategy'}
    missing = required - set(bt_df.columns)
    if missing:
        raise KeyError(f"BT CSVに必要な列がありません: {missing}")
    exit_columns = ['ticket', 'net', 'pips', 'timestamp']
    if 'exit_reason' in bt_df.columns:
        exit_columns.append('exit_reason')
    exits = bt_df[bt_df['event'] == 'EXIT'][exit_columns].copy()
    exits = exits.rename(columns={'timestamp': 'exit_timestamp', 'net': 'exit_net', 'pips': 'exit_pips'})

    merged_entries = merged.copy()
    redundant = [col for col in ('exit_reason', 'exit_timestamp', 'exit_net', 'exit_pips') if col in merged_entries.columns]
    if redundant:
        merged_entries = merged_entries.drop(columns=redundant)

    entry_exit = merged_entries.merge(exits, on='ticket', how='left')

    for col in ['atr_entry', 'adx_entry', 'donchian_width']:
        if col not in entry_exit.columns:
            entry_exit[col] = pd.NA

    width_candidates = [
        'donchian_width', 'donchian_width_x', 'donchian_width_entry', 'donchian_width_bt',
        'donchian_width_y', 'donchian_width_env', 'donchian_width_exit'
    ]
    width_series = None
    for col in width_candidates:
        if col in entry_exit.columns:
            data = entry_exit[col]
            width_series = data.copy() if width_series is None else width_series.fillna(data)
    if width_series is None:
        width_series = pd.Series(pd.NA, index=entry_exit.index)
    entry_exit['donchian_width'] = pd.to_numeric(width_series, errors='coerce')

    numeric_cols = ['atr_entry', 'adx_entry', 'donchian_width', 'exit_net', 'exit_pips']
    for col in numeric_cols:
        if col in entry_exit.columns:
            entry_exit[col] = pd.to_numeric(entry_exit[col], errors='coerce')

    if 'exit_reason' in entry_exit.columns:
        entry_exit['exit_reason'] = entry_exit['exit_reason'].fillna('UNKNOWN')
    else:
        entry_exit['exit_reason'] = 'UNKNOWN'

    return entry_exit

def ensure_entry_exit():
    notebook_globals = globals()
    entry_exit = notebook_globals.get('entry_exit')
    if entry_exit is not None:
        return entry_exit
    bt_df = notebook_globals.get('bt_df')
    merged = notebook_globals.get('merged')
    if bt_df is None or merged is None:
        raise NameError('entry_exit を生成するには bt_df と merged が必要です。')
    entry_exit = build_entry_exit(bt_df, merged)
    notebook_globals['entry_exit'] = entry_exit
    return entry_exit

ATR_BUCKET_BINS = [0, 0.10, 0.14, 0.18, 1]
ATR_BUCKET_LABELS = ['0.00-0.10','0.10-0.14','0.14-0.18','0.18+']
ADX_BUCKET_BINS = [0, 25, 30, 100]
ADX_BUCKET_LABELS = ['<=25','25-30','30+']
DONCHIAN_BUCKET_BINS = [0, 0.25, 1]
DONCHIAN_BUCKET_LABELS = ['low','high']

CURRENT_NOTEBOOK_PATH = WORKSPACE_ROOT / Path('analysis/notebooks/state_bt_analysis.ipynb')
OUTPUT_NOTEBOOK_DIR = CURRENT_NOTEBOOK_PATH.parent

pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 120)


Workspace root: /home/anyo_/workspace/YoYoEA_Multi_Entry
Env dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/env_data
BT dir: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/bt_results
Detected env csv: 195
Detected bt csv: 3


In [2]:

# ==== 解析設定 ====
BT_FILE_NAME = None  # 解析対象のBTログ。最新ファイルを使う場合は None のまま
ENV_SEPARATOR = ';'  # StateLog CSV の区切り文字
MERGE_TOLERANCE = '5min'  # ENTRY時刻と環境データを突き合わせる許容時間
SAMPLE_PREVIEW_ROWS = 5  # ロード直後に表示するサンプル行数

pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 120)


In [3]:

# ==== BT/環境データの読み込み ====
bt_df, bt_path = load_bt_dataframe(BT_FILE_NAME)
print(f'Using BT file: {bt_path.name} ({len(bt_df)} rows)')

date_keys = sorted(bt_df['timestamp'].dt.strftime('%Y%m%d').unique())
env_frames, missing_dates = load_env_frames(tuple(date_keys), sep=ENV_SEPARATOR)
if missing_dates:
    print('[WARN] 環境ファイル未検出日:', ', '.join(missing_dates))

if not env_frames:
    raise FileNotFoundError('BT対象日に対応する環境ファイルが見つかりません')

env_df = concat_env_frames(env_frames)
print('Loaded env rows:', len(env_df), 'from', len(env_frames), '日分')

merged = merge_entries_with_env(bt_df, env_df, tolerance=MERGE_TOLERANCE)
print('merged rows', len(merged))
preview_cols = [col for col in ['timestamp_x', 'event', 'strategy', 'atr_entry', 'adx_entry', 'exit_reason'] if col in merged.columns]
if preview_cols:
    sample = merged[preview_cols].head(SAMPLE_PREVIEW_ROWS).fillna('-')
    print('--- merged sample ---')
    print(sample.to_string(index=False))
else:
    print('preview columns not available')


Using BT file: TradeLog_v125_AtrBandConfig_YoYoEA_Multi_Entry_M15_MA.csv (323 rows)
Loaded env rows: 12788 from 117 日分
merged rows 162
--- merged sample ---
        timestamp_x event strategy  atr_entry adx_entry exit_reason
2025-01-02 08:45:00 ENTRY MA_CROSS   0.105214         -           -
2025-01-02 14:15:00 ENTRY MA_CROSS   0.137643         -           -
2025-01-03 16:00:00 ENTRY MA_CROSS   0.089714         -           -
2025-01-03 18:00:00 ENTRY MA_CROSS   0.118286         -           -
2025-01-03 19:45:00 ENTRY MA_CROSS   0.134929         -           -


In [4]:

# ==== ENTRY環境とEXIT結果を結合 ====
entry_exit = build_entry_exit(bt_df, merged)
print('combined rows', len(entry_exit))
preview_cols = [col for col in ['timestamp_x', 'strategy', 'atr_entry', 'adx_entry', 'exit_reason', 'exit_net'] if col in entry_exit.columns]
if preview_cols:
    sample = entry_exit[preview_cols].head(SAMPLE_PREVIEW_ROWS).fillna('-')
    print('--- entry/exit sample ---')
    print(sample.to_string(index=False))


combined rows 162
--- entry/exit sample ---
        timestamp_x strategy  atr_entry adx_entry    exit_reason  exit_net
2025-01-02 08:45:00 MA_CROSS   0.105214         - STOP_BREAKEVEN      1.91
2025-01-02 14:15:00 MA_CROSS   0.137643         -      STOP_LOSS    -22.84
2025-01-03 16:00:00 MA_CROSS   0.089714         -      STOP_LOSS    -14.80
2025-01-03 18:00:00 MA_CROSS   0.118286         -      STOP_LOSS    -19.60
2025-01-03 19:45:00 MA_CROSS   0.134929         -      STOP_LOSS    -22.29


/tmp/ipykernel_36755/2821409405.py:112: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  width_series = data.copy() if width_series is None else width_series.fillna(data)


In [5]:
# ==== セッション×指標サマリ ====
entry_exit = ensure_entry_exit()
entry_exit['exit_net'] = entry_exit['exit_net'].astype(float)

if 'session' not in entry_exit.columns:
    entry_exit['session'] = 'UNKNOWN'
else:
    entry_exit['session'] = entry_exit['session'].fillna('UNKNOWN')

if 'strategy' not in entry_exit.columns:
    raise KeyError('strategy 列が存在しません')

summary_cols = ['session', 'strategy']


def summarize(group):
    total = len(group)
    wins = group['exit_net'] > 0
    losses = group['exit_net'] < 0
    gross_profit = group.loc[wins, 'exit_net'].sum()
    gross_loss = group.loc[losses, 'exit_net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'exit_net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'exit_net'].mean() if losses.any() else 0.0
    expectancy = group['exit_net'].mean()
    return pd.Series({
        'Trades': total,
        'Wins': int(wins.sum()),
        'Losses': int(losses.sum()),
        'Win Rate (%)': win_rate * 100,
        'PF': profit_factor,
        'Avg Win': avg_win,
        'Avg Loss': avg_loss,
        'Expectancy': expectancy,
    })

session_strategy = (
    entry_exit.groupby(summary_cols, group_keys=False, observed=False)
    .apply(summarize, include_groups=False)
    .reset_index()
    .sort_values(['session', 'strategy'])
)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}

session_strategy.style.format(format_dict).set_table_attributes('style="table-layout: fixed; width: 100%;"')


,session,strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
0,ASIA,MA_CROSS,34.000000,27.000000,7.000000,79.41,2.84,14.88,-20.21,7.66
1,EUROPE,MA_CROSS,42.000000,24.000000,18.000000,57.14,1.05,17.16,-21.86,0.44
2,OTHER,MA_CROSS,19.000000,10.000000,9.000000,52.63,0.51,14.89,-32.38,-7.50
3,US,MA_CROSS,67.000000,43.000000,23.000000,64.18,0.97,11.98,-23.19,-0.27


In [6]:
# ==== ATR×ADX帯域の損益集計 ====
entry_exit = ensure_entry_exit()
attr = entry_exit.dropna(subset=['atr_entry','strategy']).copy()
adx_candidates = []
if 'adx_entry' in attr.columns:
    adx_candidates.append(attr['adx_entry'])
if 'adx14' in attr.columns:
    adx_candidates.append(attr['adx14'])
if not adx_candidates:
    raise ValueError('ADX列が見つかりません (adx_entry/ adx14)')
adx_value = None
for series in adx_candidates:
    adx_value = series if adx_value is None else adx_value.combine_first(series)
attr['adx_value'] = adx_value
attr = attr.dropna(subset=['adx_value'])
if attr.empty:
    raise ValueError('ATR/ADX/strategy が不足しています')
attr['atr_bucket'] = pd.cut(attr['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
attr['adx_bucket'] = pd.cut(attr['adx_value'].astype(float), bins=ADX_BUCKET_BINS, labels=ADX_BUCKET_LABELS, right=False)
metrics = attr.pivot_table(index=['atr_bucket','adx_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['count', :]] = metrics.loc[:, idx['count', :]].astype(int)
if 'sum' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['sum', :]] = metrics.loc[:, idx['sum', :]].round(2)
if 'mean' in metrics.columns.get_level_values(0):
    metrics.loc[:, idx['mean', :]] = metrics.loc[:, idx['mean', :]].round(2)
metrics.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [7]:
# ==== ATR帯×期間ばらつきサマリ ====
entry_exit = ensure_entry_exit()
exits = entry_exit.dropna(subset=['atr_entry','exit_net']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')

# 期間キー（優先: exit_timestamp -> timestamp_x -> timestamp）
ts_col = None
for cand in ('exit_timestamp', 'timestamp_x', 'timestamp'):
    if cand in exits.columns:
        ts_col = cand
        break
if ts_col is None:
    raise KeyError('時刻列が見つかりません (exit_timestamp/timestamp_x/timestamp)')

exits[ts_col] = pd.to_datetime(exits[ts_col])
exits['period'] = exits[ts_col].dt.to_period('M').dt.to_timestamp()
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)

def summarize_period(g):
    total = len(g)
    wins = g['exit_net'] > 0
    losses = g['exit_net'] < 0
    gross_profit = g.loc[wins, 'exit_net'].sum()
    gross_loss = g.loc[losses, 'exit_net'].sum()
    pf = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    expectancy = g['exit_net'].mean()
    return pd.Series({
        'trades': total,
        'win_rate_pct': win_rate * 100,
        'pf': pf,
        'expectancy': expectancy,
        'sum_net': g['exit_net'].sum(),
    })

period_stats = (
    exits.groupby(['atr_bucket','period'], observed=False)
    .apply(summarize_period)
    .reset_index()
    .sort_values(['atr_bucket','period'])
)

# 期間方向の分位点でばらつきを確認
band_quantiles = period_stats.groupby('atr_bucket', observed=False).agg({
    'trades': ['mean', 'median'],
    'win_rate_pct': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'pf': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'expectancy': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
})
band_quantiles.columns = [
    'trades_mean', 'trades_median',
    'win_rate_median', 'win_rate_p25', 'win_rate_p75',
    'pf_median', 'pf_p25', 'pf_p75',
    'expectancy_median', 'expectancy_p25', 'expectancy_p75',
]
band_quantiles = band_quantiles.round(2)
print('--- period_stats sample ---')
print(period_stats.head())
band_quantiles


--- period_stats sample ---
  atr_bucket     period  trades  win_rate_pct        pf  expectancy  sum_net
0  0.00-0.10 2025-01-01     3.0     33.333333  0.979730      -0.200    -0.60
1  0.00-0.10 2025-02-01     5.0     60.000000  1.180344       1.112     5.56
2  0.00-0.10 2025-03-01     2.0      0.000000  0.000000     -14.965   -29.93
3  0.00-0.10 2025-04-01     2.0     50.000000  0.124301      -7.045   -14.09
4  0.00-0.10 2025-05-01     2.0    100.000000       inf       2.065     4.13


/tmp/ipykernel_36755/254438320.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_period)
/home/anyo_/workspace/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,trades_mean,trades_median,win_rate_median,win_rate_p25,win_rate_p75,pf_median,pf_p25,pf_p75,expectancy_median,expectancy_p25,expectancy_p75
atr_bucket,,,,,,,,,,,
0.00-0.10,4.12,3.0,56.92,45.83,66.67,1.08,0.67,2.00,0.46,-2.51,2.55
0.10-0.14,7.88,7.5,70.71,64.58,81.25,1.33,0.84,inf,1.78,-1.54,9.25
0.14-0.18,4.43,4.0,57.14,50.00,70.83,0.74,0.07,2.36,-3.06,-12.37,10.51
0.18+,4.25,3.5,87.50,65.00,100.00,inf,0.98,NaN,2.57,-0.81,11.19


In [8]:

# ==== ATR帯×期間ばらつき（戦略別） ====
entry_exit = ensure_entry_exit()
exits = entry_exit.dropna(subset=['atr_entry','exit_net','strategy']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')

# 期間キー（優先: exit_timestamp -> timestamp_x -> timestamp）
ts_col = None
for cand in ('exit_timestamp', 'timestamp_x', 'timestamp'):
    if cand in exits.columns:
        ts_col = cand
        break
if ts_col is None:
    raise KeyError('時刻列が見つかりません (exit_timestamp/timestamp_x/timestamp)')

exits[ts_col] = pd.to_datetime(exits[ts_col])
exits['period'] = exits[ts_col].dt.to_period('M').dt.to_timestamp()
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)


def summarize_period(g):
    total = len(g)
    wins = g['exit_net'] > 0
    losses = g['exit_net'] < 0
    gross_profit = g.loc[wins, 'exit_net'].sum()
    gross_loss = g.loc[losses, 'exit_net'].sum()
    pf = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    expectancy = g['exit_net'].mean()
    return pd.Series({
        'trades': total,
        'win_rate_pct': win_rate * 100,
        'pf': pf,
        'expectancy': expectancy,
        'sum_net': g['exit_net'].sum(),
    })

period_stats_s = (
    exits.groupby(['atr_bucket','strategy','period'], observed=False)
    .apply(summarize_period)
    .reset_index()
    .sort_values(['atr_bucket','strategy','period'])
)

band_quantiles_s = period_stats_s.groupby(['atr_bucket','strategy'], observed=False).agg({
    'trades': ['mean', 'median'],
    'win_rate_pct': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'pf': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
    'expectancy': ['median', lambda s: s.quantile(0.25), lambda s: s.quantile(0.75)],
})
band_quantiles_s.columns = [
    'trades_mean', 'trades_median',
    'win_rate_median', 'win_rate_p25', 'win_rate_p75',
    'pf_median', 'pf_p25', 'pf_p75',
    'expectancy_median', 'expectancy_p25', 'expectancy_p75',
]
band_quantiles_s = band_quantiles_s.round(2)
print('--- period_stats_s sample ---')
print(period_stats_s.head())
band_quantiles_s



--- period_stats_s sample ---
  atr_bucket  strategy     period  trades  win_rate_pct        pf  expectancy  sum_net
0  0.00-0.10  MA_CROSS 2025-01-01     3.0     33.333333  0.979730      -0.200    -0.60
1  0.00-0.10  MA_CROSS 2025-02-01     5.0     60.000000  1.180344       1.112     5.56
2  0.00-0.10  MA_CROSS 2025-03-01     2.0      0.000000  0.000000     -14.965   -29.93
3  0.00-0.10  MA_CROSS 2025-04-01     2.0     50.000000  0.124301      -7.045   -14.09
4  0.00-0.10  MA_CROSS 2025-05-01     2.0    100.000000       inf       2.065     4.13


/tmp/ipykernel_36755/3537096212.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_period)
/home/anyo_/workspace/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,,trades_mean,trades_median,win_rate_median,win_rate_p25,win_rate_p75,pf_median,pf_p25,pf_p75,expectancy_median,expectancy_p25,expectancy_p75
atr_bucket,strategy,,,,,,,,,,,
0.00-0.10,MA_CROSS,4.12,3.0,56.92,45.83,66.67,1.08,0.67,2.00,0.46,-2.51,2.55
0.10-0.14,MA_CROSS,7.88,7.5,70.71,64.58,81.25,1.33,0.84,inf,1.78,-1.54,9.25
0.14-0.18,MA_CROSS,4.43,4.0,57.14,50.00,70.83,0.74,0.07,2.36,-3.06,-12.37,10.51
0.18+,MA_CROSS,4.25,3.5,87.50,65.00,100.00,inf,0.98,NaN,2.57,-0.81,11.19


In [9]:

# ==== ATR×Donchian帯域の損益集計 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width','strategy']).copy()
if don.empty:
    raise ValueError('ATR/Donchian/strategy が不足しています')
don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=DONCHIAN_BUCKET_BINS, labels=DONCHIAN_BUCKET_LABELS, right=False)
metrics_d = don.pivot_table(index=['atr_bucket','donchian_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
idx = pd.IndexSlice
if 'count' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['count', :]] = metrics_d.loc[:, idx['count', :]].astype(int)
if 'sum' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['sum', :]] = metrics_d.loc[:, idx['sum', :]].round(2)
if 'mean' in metrics_d.columns.get_level_values(0):
    metrics_d.loc[:, idx['mean', :]] = metrics_d.loc[:, idx['mean', :]].round(2)
metrics_d.style     .format('{:.2f}', subset=idx[:, idx[['sum','mean'], :]])     .format('{:d}', subset=idx[:, idx[['count'], :]])     .set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [10]:
# ==== ATR帯×戦略×EXIT理由の損益 ====
entry_exit = ensure_entry_exit()
if 'exit_reason' not in entry_exit.columns:
    entry_exit['exit_reason'] = 'UNKNOWN'
exits = entry_exit.dropna(subset=['atr_entry','strategy','exit_net']).copy()
if exits.empty:
    raise ValueError('EXIT結果が存在しません')
exits['exit_reason'] = exits['exit_reason'].fillna('UNKNOWN')
exits['atr_bucket'] = pd.cut(exits['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
exit_summary = (
    exits.groupby(['atr_bucket','strategy','exit_reason'], observed=False)
    .agg(count=('exit_net','size'), sum=('exit_net','sum'), mean=('exit_net','mean'))
    .reset_index()
    .sort_values(['atr_bucket','strategy','exit_reason'])
)
exit_summary[['sum','mean']] = exit_summary[['sum','mean']].round(2)
exit_summary['count'] = exit_summary['count'].astype(int)
exit_summary


,atr_bucket,strategy,exit_reason,count,sum,mean
0,0.00-0.10,MA_CROSS,STOP_BREAKEVEN,12,24.34,2.03
1,0.00-0.10,MA_CROSS,STOP_LOSS,15,-224.56,-14.97
2,0.00-0.10,MA_CROSS,STOP_TRAILING,0,0.00,NaN
3,0.00-0.10,MA_CROSS,TAKE_PROFIT,6,187.84,31.31
4,0.10-0.14,MA_CROSS,STOP_BREAKEVEN,30,60.43,2.01
5,0.10-0.14,MA_CROSS,STOP_LOSS,22,-452.85,-20.58
6,0.10-0.14,MA_CROSS,STOP_TRAILING,0,0.00,NaN
7,0.10-0.14,MA_CROSS,TAKE_PROFIT,11,499.20,45.38
8,0.14-0.18,MA_CROSS,STOP_BREAKEVEN,14,28.09,2.01
9,0.14-0.18,MA_CROSS,STOP_LOSS,12,-329.19,-27.43


In [11]:
# ==== ATR帯×戦略×EXIT理由（合計ピボット） ====
if 'exit_summary' not in globals():
    raise NameError('exit_summary が見つかりません (前セルを実行してください)')
exit_pivot = (
    exit_summary
    .pivot_table(index=['atr_bucket','strategy'], columns='exit_reason', values='sum', fill_value=0, observed=False)
    .sort_index()
    .round(2)
)
exit_pivot


,exit_reason,STOP_BREAKEVEN,STOP_LOSS,STOP_TRAILING,TAKE_PROFIT
atr_bucket,strategy,,,,
0.00-0.10,MA_CROSS,24.34,-224.56,0.00,187.84
0.10-0.14,MA_CROSS,60.43,-452.85,0.00,499.20
0.14-0.18,MA_CROSS,28.09,-329.19,0.00,287.65
0.18+,MA_CROSS,14.79,-353.00,375.56,0.00


In [12]:

# ==== Donchian×ATR×ストラテジーの損益 ====
entry_exit = ensure_entry_exit()
don = entry_exit.dropna(subset=['atr_entry','donchian_width','strategy']).copy()
if don.empty:
    print('Donchian幅を持つレコードがありません')
else:
    don['atr_bucket'] = pd.cut(don['atr_entry'].astype(float), bins=ATR_BUCKET_BINS, labels=ATR_BUCKET_LABELS, right=False)
    don['donchian_bucket'] = pd.cut(don['donchian_width'], bins=DONCHIAN_BUCKET_BINS, labels=DONCHIAN_BUCKET_LABELS, right=False)
    table_d = don.pivot_table(index=['atr_bucket','donchian_bucket'], columns='strategy', values='exit_net', aggfunc=['count','sum','mean'], fill_value=0, observed=False)
    idx = pd.IndexSlice
    if 'count' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['count', :]] = table_d.loc[:, idx['count', :]].astype(int)
    if 'sum' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['sum', :]] = table_d.loc[:, idx['sum', :]].round(2)
    if 'mean' in table_d.columns.get_level_values(0):
        table_d.loc[:, idx['mean', :]] = table_d.loc[:, idx['mean', :]].round(2)

    style = table_d.style
    value_levels = [lvl for lvl in ('sum','mean') if lvl in table_d.columns.get_level_values(0)]
    if value_levels:
        style = style.format('{:.2f}', subset=idx[:, idx[value_levels, :]])
    if 'count' in table_d.columns.get_level_values(0):
        style = style.format('{:d}', subset=idx[:, idx[['count'], :]])
    style.set_table_attributes('style="table-layout: fixed; width: 100%;"')


In [13]:
# ==== 例: ATR帯域×戦略の損益集計 ====
exits = bt_df[bt_df['event'] == 'EXIT'].copy()
exits['atr_entry'] = exits['atr_entry'].astype(float)
bins = ATR_BUCKET_BINS
labels = ATR_BUCKET_LABELS
exits['atr_band'] = pd.cut(exits['atr_entry'], bins=bins, labels=labels, right=False)
pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0, observed=False)
pivot = pivot.round(2)
pivot.style.format('{:.2f}').set_table_attributes('style="table-layout: fixed; width: 100%;"')


strategy,MA_CROSS
atr_band,
0.00-0.10,-12.38
0.10-0.14,106.78
0.14-0.18,-13.45
0.18+,37.35


In [14]:
# ==== BT全体指標のサマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')
bt_exits = bt_exits.sort_values('timestamp')

bt_exits['net'] = bt_exits['net'].astype(float)
bt_exits['pips'] = bt_exits['pips'].astype(float)

total_trades = len(bt_exits)
wins = bt_exits['net'] > 0
losses = bt_exits['net'] < 0

gross_profit = bt_exits.loc[wins, 'net'].sum()
gross_loss = bt_exits.loc[losses, 'net'].sum()
profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
win_rate = wins.sum() / total_trades if total_trades else float('nan')

avg_win = bt_exits.loc[wins, 'net'].mean() if wins.any() else 0.0
avg_loss = bt_exits.loc[losses, 'net'].mean() if losses.any() else 0.0
expectancy = bt_exits['net'].mean()

# 最大ドローダウンの算出
equity_curve = bt_exits['net'].cumsum()
equity_curve = pd.concat([pd.Series([0.0]), equity_curve], ignore_index=True)
rolling_peak = equity_curve.cummax()
drawdown = equity_curve - rolling_peak
max_drawdown = drawdown.min()
drawdown_pct = drawdown / rolling_peak.replace(0, pd.NA)
max_drawdown_pct = drawdown_pct.min()

summary = {
    'trades': total_trades,
    'wins': int(wins.sum()),
    'losses': int(losses.sum()),
    'win_rate_pct': win_rate * 100,
    'gross_profit': gross_profit,
    'gross_loss': gross_loss,
    'profit_factor': profit_factor,
    'avg_win': avg_win,
    'avg_loss': avg_loss,
    'expectancy': expectancy,
    'max_drawdown': max_drawdown,
    'max_drawdown_pct': max_drawdown_pct * 100,
}

label_map = {
    'trades': '総トレード数',
    'wins': '勝ち回数',
    'losses': '負け回数',
    'win_rate_pct': '勝率',
    'gross_profit': '総利益',
    'gross_loss': '総損失',
    'profit_factor': 'PF',
    'avg_win': '平均利益',
    'avg_loss': '平均損失',
    'expectancy': '期待値',
    'max_drawdown': '最大DD',
    'max_drawdown_pct': '最大DD(%)',
}
percent_keys = {'win_rate_pct', 'max_drawdown_pct'}
value_keys = {'profit_factor', 'avg_win', 'avg_loss', 'expectancy', 'gross_profit', 'gross_loss', 'max_drawdown'}

rows = []
for key in label_map:
    value = summary.get(key)
    if pd.isna(value):
        display_value = 'NaN'
    elif key in percent_keys:
        display_value = f"{value:.2f}%"
    elif key in value_keys:
        display_value = f"{value:.2f}"
    else:
        display_value = f"{int(value)}"
    rows.append({'指標': label_map[key], '値': display_value})

summary_df = pd.DataFrame(rows)
summary_df


,指標,値
0,総トレード数,161
1,勝ち回数,104
2,負け回数,57
3,勝率,64.60%
4,総利益,1477.90
5,総損失,-1359.60
6,PF,1.09
7,平均利益,14.21
8,平均損失,-23.85
9,期待値,0.73


In [15]:
# ==== 戦略別BT指標サマリ ====
bt_exits = bt_df[bt_df['event'] == 'EXIT'].copy()
if bt_exits.empty:
    raise ValueError('EXITデータが存在しません')
if 'strategy' not in bt_exits.columns:
    raise KeyError('strategy 列が存在しません')

bt_exits['net'] = bt_exits['net'].astype(float)

def summarize_group(group):
    total = len(group)
    wins = group['net'] > 0
    losses = group['net'] < 0
    gross_profit = group.loc[wins, 'net'].sum()
    gross_loss = group.loc[losses, 'net'].sum()
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')
    win_rate = wins.sum() / total if total else float('nan')
    avg_win = group.loc[wins, 'net'].mean() if wins.any() else 0.0
    avg_loss = group.loc[losses, 'net'].mean() if losses.any() else 0.0
    expectancy = group['net'].mean()
    return pd.Series({
        'trades': total,
        'wins': int(wins.sum()),
        'losses': int(losses.sum()),
        'win_rate_pct': win_rate * 100,
        'profit_factor': profit_factor,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'expectancy': expectancy,
    })

strategy_summary = (
    bt_exits.groupby('strategy', group_keys=False)
    .apply(summarize_group)
    .reset_index()
)

rename_map = {
    'strategy': 'Strategy',
    'trades': 'Trades',
    'wins': 'Wins',
    'losses': 'Losses',
    'win_rate_pct': 'Win Rate (%)',
    'profit_factor': 'PF',
    'avg_win': 'Avg Win',
    'avg_loss': 'Avg Loss',
    'expectancy': 'Expectancy',
}
strategy_summary = strategy_summary.rename(columns=rename_map)

format_dict = {
    'Win Rate (%)': '{:.2f}',
    'PF': '{:.2f}',
    'Avg Win': '{:.2f}',
    'Avg Loss': '{:.2f}',
    'Expectancy': '{:.2f}',
}
strategy_summary.style.format(format_dict).hide(axis="index")


/tmp/ipykernel_36755/1671446051.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_group)


Strategy,Trades,Wins,Losses,Win Rate (%),PF,Avg Win,Avg Loss,Expectancy
MA_CROSS,161.000000,104.000000,57.000000,64.60,1.09,14.21,-23.85,0.73


## TODO
- 週次/日次での State×BT 結合関数を関数化する
- 追加の特徴量（ATR増減、セッションフラグなど）を派生列として定義する
- ML モデル用に `merged` から特徴量テーブルを作成する

In [16]:
# ==== ノート別名保存（BTファイル名付き） ====
import nbformat
if 'bt_path' not in globals():
    raise NameError('bt_path が未定義です (データ読み込みセルを先に実行してください)')
base_name = bt_path.stem.replace(' ', '_')
dest_path = OUTPUT_NOTEBOOK_DIR / f"state_bt_analysis_{base_name}.ipynb"
with CURRENT_NOTEBOOK_PATH.open('r', encoding='utf-8') as fh:
    nb_data = nbformat.read(fh, as_version=4)
nbformat.write(nb_data, str(dest_path))
print(f'Saved notebook copy to: {dest_path}')


Saved notebook copy to: /home/anyo_/workspace/YoYoEA_Multi_Entry/analysis/notebooks/state_bt_analysis_TradeLog_v125_AtrBandConfig_YoYoEA_Multi_Entry_M15_MA.ipynb
